In [ ]:
from google.colab import drive
from bs4 import BeautifulSoup
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install langdetect
!pip install translate
from translate import Translator
from langdetect import detect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=3076159ac129137c9e7cf0375562cb745b320cef3056b429d96fdb3db42e76ce
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import pandas as pd
train_data = pd.read_csv("/content/drive/My Drive/IR/Project/Job_Posting_dataset.csv")
test_data = pd.read_csv("/content/drive/My Drive/IR/Project/testing_dataset.csv")
test_data

,LinkedIn Resume,Job Profiles
0,Shyvee Shi\n(She/Her)\n3rd degree connection\n...,"['Product Manager','Business Analyst','UX Desi..."
1,Elisa Bellagamba\n\nAbout\nHigh-impact product...,"['Product Manager','Business Analyst','Marketi..."
2,Joni (Rafalski) Hoadley (She/Her)\n\nAbout\nI’...,"['Product Manager','UX Designer','Business Ana..."
3,Shane Connelly\n\nAbout\nI lead the product ma...,"['Product Manager','Software Developer,'Machin..."
4,Dana Tom\n(She/Her)\n\nAbout\nI’m a product ma...,"['Product Manager','Marketing Analyst','Machin..."
...,...,...
367,Johanna P.\n\nAbout\nExperienced Bilingual Rep...,"['Sales Analyst','Marketing Analyst,'Business ..."
368,Rachel Sedlacek\n\nAbout\nAnalyst with 12 year...,"['Sales Analyst','Marketing Analyst,'Business ..."
369,Deirdre Derby\n\nAbout\n[]\n\nLocation\nMinnea...,"['Sales Analyst','Marketing Analyst,'Business ..."
370,Ryan Mitchell\n\nAbout\nSr. Sales Analyst for ...,"['Sales Analyst','Marketing Analyst,'Business ..."


In [ ]:
#@title Removing all the rows that contain description in other languages than English
for index, row in train_data.iterrows():
    try:
        lang = detect(row['Job_Description'])
        if lang != 'en':
            train_data.drop(index, inplace=True)
    except:
        # if an error occurs, assume the language is not English and drop the row
        train_data.drop(index, inplace=True)
train_data.reset_index(inplace=True)
train_data

,index,Job ID,Job Title,Company,Job_Description,Job Profile
0,1,3489403427,Software Engineer,LinkedIn,"The ideal candidate will help build, maintain,...",Software Developer
1,2,3490979195,Software Engineer,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",Software Developer
2,3,3507663809,Junior Software Developer (Web/Front-End),Samsung Brasil,Position Summary\n\n\n\n\nDevelop a differenti...,Software Developer
3,4,3497871312,Software Engineer,Oracle,Want to come join the Oracle Health Data & Ana...,Software Developer
4,6,3497654432,Software Engineer,Illuma,We are looking for a highly motivated Software...,Software Developer
...,...,...,...,...,...,...
909,985,3496041048,Marketing Analyst,TI Fluid Systems,Description\n\nDescription:\n\nPosition Summar...,Marketing Analyst
910,986,3496037782,Marketing Analyst,TI Fluid Systems,Description\n\nDescription:\n\nPosition Summar...,Marketing Analyst
911,987,3485523798,Marketing Analyst,Hose Master LLC,POSITION SUMMARY\n\nThe Marketing Analyst is r...,Marketing Analyst
912,988,3488196177,Senior Marketing Analyst,Maxeon Solar Technologies,Maxeon Solar Technologies is a global leader i...,Marketing Analyst


In [ ]:
#@title Removing all the rows that contain description in other languages than English
for index, row in test_data.iterrows():
    try:
        lang = detect(row['LinkedIn Resume'])
        if lang != 'en':
            test_data.drop(index, inplace=True)
    except:
        # if an error occurs, assume the language is not English and drop the row
        test_data.drop(index, inplace=True)
test_data.reset_index(inplace=True)
test_data

,index,LinkedIn Resume,Job Profiles
0,0,Shyvee Shi\n(She/Her)\n3rd degree connection\n...,"['Product Manager','Business Analyst','UX Desi..."
1,1,Elisa Bellagamba\n\nAbout\nHigh-impact product...,"['Product Manager','Business Analyst','Marketi..."
2,2,Joni (Rafalski) Hoadley (She/Her)\n\nAbout\nI’...,"['Product Manager','UX Designer','Business Ana..."
3,3,Shane Connelly\n\nAbout\nI lead the product ma...,"['Product Manager','Software Developer,'Machin..."
4,4,Dana Tom\n(She/Her)\n\nAbout\nI’m a product ma...,"['Product Manager','Marketing Analyst','Machin..."
...,...,...,...
367,367,Johanna P.\n\nAbout\nExperienced Bilingual Rep...,"['Sales Analyst','Marketing Analyst,'Business ..."
368,368,Rachel Sedlacek\n\nAbout\nAnalyst with 12 year...,"['Sales Analyst','Marketing Analyst,'Business ..."
369,369,Deirdre Derby\n\nAbout\n[]\n\nLocation\nMinnea...,"['Sales Analyst','Marketing Analyst,'Business ..."
370,370,Ryan Mitchell\n\nAbout\nSr. Sales Analyst for ...,"['Sales Analyst','Marketing Analyst,'Business ..."


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Define stop words
stop_words = set(stopwords.words('english'))

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(words)
    return text

# Apply preprocess_text() function to job description column
train_data['Job_Description'] = train_data['Job_Description'].apply(preprocess_text)
test_data['LinkedIn Resume'] = test_data['LinkedIn Resume'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
train_data.head()

,index,Job ID,Job Title,Company,Job_Description,Job Profile
0,1,3489403427,Software Engineer,LinkedIn,"ideal candidate help build , maintain , troubl...",Software Developer
1,2,3490979195,Software Engineer,PayPal,"paypal ( nasdaq : pypl ) , believe every perso...",Software Developer
2,3,3507663809,Junior Software Developer (Web/Front-End),Samsung Brasil,position summary develop differentiated system...,Software Developer
3,4,3497871312,Software Engineer,Oracle,want come join oracle health data & analytics ...,Software Developer
4,6,3497654432,Software Engineer,Illuma,looking highly motivated software engineer wor...,Software Developer


In [ ]:
def getmappings(le):
  d=dict(zip(le.classes_, le.transform(le.classes_)))
  revd=dict(zip(le.classes_, le.transform(le.classes_)))
  return d, revd
def encodetest(df,le):
  d, revd=getmappings(le)
  print(d)
  lst=df.at[0, 'Job Profiles']
  for i in range(len(df['Job Profiles'])):
    st=df.at[i, 'Job Profiles']
    st = st.replace("'","")
    st = st.replace("[","")
    st = st.replace("]","")
    lst = st.split(',')
    # lst=ast.literal_eval(lst)
    for j in range(len(lst)):
      # print(lst[2])
      lst[j]=int(d[lst[j]])
    df.at[i, 'Job Profiles']=lst

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['Job Profile'] = le.fit_transform(train_data['Job Profile'])
encodetest(test_data,le)

{'Account Manager': 0, 'Business Analyst': 1, 'Machine Learning Engineer': 2, 'Marketing Analyst': 3, 'Product Manager': 4, 'Sales Analyst': 5, 'Software Developer': 6, 'UX Designer': 7}


**COUNT VECTORIZER (BASELINE)**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Instantiate a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data
vectorizer.fit(train_data['Job_Description'])

# Transform the job descriptions into numerical features using TF-IDF algorithm
X_train = vectorizer.transform(train_data['Job_Description'])
X_test = vectorizer.transform(test_data['LinkedIn Resume'])
y_train = train_data['Job Profile']


# Extract the class labels from the "job_profile" column
y_test = test_data["Job Profiles"].values

In [ ]:
X_train.

<914x12083 sparse matrix of type '<class 'numpy.int64'>'
	with 198655 stored elements in Compressed Sparse Row format>

**TF-IDF VECTORIZER MODEL (INCLUDED IN MID-SEM PROJECT REVIEW)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data
vectorizer.fit(train_data['Job_Description'])

# Transform the job descriptions into numerical features using TF-IDF algorithm
X_train = vectorizer.transform(train_data['Job_Description'])
X_test = vectorizer.transform(test_data['LinkedIn Resume'])
y_train = train_data['Job Profile']


# Extract the class labels from the "job_profile" column
y_test = test_data["Job Profiles"].values

**PRE-TRAINED BERT MODEL (NOT INCLUDED IN MID-SEM PROJECT REVIEW)**

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load the pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
cnt = 0
# Define a function to extract features from the job descriptions
def get_features(text):
    # Truncate the input text if it's too long
    max_input_length = 512
    if len(text) > max_input_length:
        text = text[:max_input_length]
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs[0].squeeze(0)
    global cnt
    print(cnt)
    cnt = cnt+1
    return last_hidden_states.mean(dim=0).tolist()
# Extract the features from the job descriptions in the training data
X_train = [get_features(text) for text in train_data['Job_Description']]
X_train = pd.DataFrame(X_train)

# Extract the features from the job descriptions in the testing data
X_test = [get_features(text) for text in test_data['LinkedIn Resume']]
X_test = pd.DataFrame(X_test)

# Extract the labels from the dataset
y_train = train_data['Job Profile']
y_test = test_data['Job Profiles']

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
estimators = [('rf', RandomForestClassifier(random_state=42)), ('xgb', XGBClassifier(random_state=42))]
clf = RandomForestClassifier(random_state=42)
# clf = StackingClassifier(
#     estimators=estimators, final_estimator=LogisticRegression()
# )

# clf.fit(X_train, X_test).score(X_test, y_test)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

sorted_probs = (-y_pred).argsort()
k = 3
#getting top k classes with the highest probabilities
top_k = [sorted_probs[i, :k] for i in range(len(sorted_probs))]   
print(y_pred[0])
print(sorted_probs[0])
print(top_k[0])
#Applying the formula for Precision@k
correct = [len(intersection(y_test[i][:k], top_k[i])) for i in range(len(y_test))]
precision = (np.sum(correct))/(len(y_test)*k)
print("Precision@k: " +str(precision))

[0.07 0.12 0.09 0.14 0.26 0.07 0.05 0.2 ]
[4 7 3 1 2 0 5 6]
[4 7 3]
Precision@k: 0.6451612903225806


In [ ]:
#K can be any value between 1 and 3 including both.
sorted_probs = (-y_pred).argsort()
k = 2
top_k = [sorted_probs[i, :k] for i in range(len(sorted_probs))]   
# print(y_pred[0])
# print(sorted_probs[0])
# print(top_k[0])

#Applying the formula for Recall@k
correct = [len(intersection(y_test[i], top_k[i])) for i in range(len(y_test))]
precision = np.sum(correct) / (len(y_test) * k)
print("Recall@k: " +str(precision))

Recall@k: 0.7634408602150538


In [ ]:
def instancePrecision(k,i):
  correct=0
  if(k<=3):
    correct=len(intersection(y_test[i][:k], sorted_probs[i][:k]))
  else:
    correct=len(intersection(y_test[i], sorted_probs[i][:k]))
  prec=correct/k
  return prec

#Finding Precision at all K<=3 at a particular instance.
precision3=[instancePrecision(3,i) for i in range(len(y_test))]
precision2=[instancePrecision(2,i) for i in range(len(y_test))]
precision1=[instancePrecision(1,i) for i in range(len(y_test))]
precisionAll=[[instancePrecision(1,i),instancePrecision(2,i),instancePrecision(3,i)] for i in range(len(y_test))]

#taking sum and mean.
meansum=0
for i in range(len(precision3)):
  sum=0
  for j in range(len(precisionAll[i])):
    sum+=precisionAll[i][j]
  mean=sum/len(precisionAll[i])
  meansum=meansum+mean

meanAvgprec=meansum/len(y_test)
print("Mean Average Precision: " +str(meanAvgprec))

Mean Average Precision: 0.6805555555555561
